# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Run analysis](#run) shall be run each time a new Sima analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.

### Postprocessing
It is now possible to read the SIN/FEM file using SifIO, please consult the [Python examples](https://test.pypi.org/project/dnv-sifio/) C# [documentation](https://sesam.dnv.com/dev/api/sifio/). This notebook provides a small example script for getting node coordinates and displacements for the 200 first nodes for loadcase 11, [postprocessing.py](Workspace/CommonFiles/postprocessing.py). The script will be run just after Wasim and Sestra for each load case, see the [Run Wasim and Sestra](#runwasim) section. The section [Post processing](#postprocessing) shows how the post processed result files may be read and visualized in this notebook. More Python examples will be provided later. Relevant additional documents are:
- The input interface file documentations can be found [here](https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf)
- The results interface format [here](https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf)



# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up custom user parameters <a id='custom'></a>
You need to set oneWorkflowTMPFolder to a folder that works for you. A short folder path  is recommended due to possible issues with Sima and long file names.

In [ ]:
from pathlib import Path
import os

oneWorkflowTMPFolder = r'd:\oneworkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
        print(oneWorkflowTMPFolder + " created!\n")
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath
root_folder = os.getcwd()
print(root_folder)
workspacePath = Path(root_folder, 'Workspace')
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloudRun = False
notebook_root_folder = os.getcwd()

## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [ ]:
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils.starter import *

workflow_client = one_workflow_client(workspace_id = workspaceId, workspace_path = workspacePath, cloud_run = cloudRun,
                                      local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder, platform=Platform.WINDOWS, max_cores=5,auto_deploy_option = AutoDeployOption.DEV)
workspace = workflow_client.workspace_info
commonfiles_folder = workspace.common_files_directory
results_folder = workspace.results_directory
#If running locally the code below will also start the local workflow host.
if (cloudRun):
    workflow_client.login()


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [ ]:

from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


# Run Sima analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [ ]:
from SimaHelper import *
import json
"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

#Sima path must be specified
sima_settings = SimaSettings(sima_exe_path=r'C:\Program Files\DNV\Sima V4.4-00')
sima_work_unit = SimaTaskCreator(sima_settings, workflow_client).get_sima_work_unit(loadcase_file, stask_file)

# Create Parallel Work Unit and Job
job = workflow_client.create_job(sima_work_unit)
job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)

#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    max_size="11124MB",
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg"])
await run_workflow_async(job, workflow_client, downloadOptions)


# Run Wasim and Sestra <a id='runwasim'></a>

In [ ]:
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
import json
import os
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        load_case_result_files_dir = os.path.join(results_directory, casename)
        casedict = case.to_dict()
        
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)
print(os.getcwd())
print(workflow_client.workspace_path)

work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)
if not cloudRun:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()


downloadOptions = FileOptions(
    min_size =0,
    max_size= "11124MB",
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
await run_workflow_async(workflow_client.create_job(work_unit), workflow_client, downloadOptions)



# Post processing  <a id='postprocessing'></a>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
print(workspacePath)
for folder in glob.glob(f"{workspacePath}\\Results\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)

def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(multiplot, resultcase=dataFrames.keys())
interactive_plot
